In [4]:
import tensorflow as tf
import numpy as np

/Users/monk/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/monk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [49]:
tf.reset_default_graph()

class lstm_Attention_with_Pos(object):
    
    
    def __init__(self,vocab_size,embedding_dim,num_cell,dropout_value,pos_vocab_size,pdim,category_size):
        
        
        
        
        sentence = tf.placeholder(name='input_data',shape=[None,None],dtype=tf.int32)
        pos_     = tf.placeholder(name='pos',shape=[None,None],dtype=tf.int32)
        intents  = tf.placeholder(name='labels',shape=[None],dtype=tf.int32)
        mode =  tf.placeholder(name='dropout',shape=(),dtype=tf.int32)
        
        
        
        
        
        self.placeholders = {
            
                            'input_d':sentence,
                            'labels':intents,
                             'pos':pos_,
                            'dropout':mode
                            }
        

        
        
        embedding_matrix = tf.get_variable(name='word_embedding',
                                           shape=[vocab_size,embedding_dim],
                                           dtype=tf.float32,
                                           initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        embedd_word2vec = tf.nn.embedding_lookup(embedding_matrix,sentence)
        
        dropout = tf.cond(
                          tf.equal(mode,0),
                          lambda : dropout_value, 
                          lambda : 0.
                         )
        
        pos_embedding = tf.get_variable(name='pos_embedding',
                                           shape=[pos_vocab_size,pdim],
                                           dtype=tf.float32,
                                           initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
        pos_lookup = tf.nn.embedding_lookup(pos_embedding, pos_ )
        
        final_input = tf.concat([embedd_word2vec,pos_lookup],axis=-1)
        
        
                
        
        
        
        
        sequence_length = tf.count_nonzero(sentence,axis=-1)
        
        
         
        lstm_cell_fw = tf.contrib.rnn.LSTMCell(num_cell,forget_bias=1.0)
        dropout_wrapper_fw = tf.contrib.rnn.DropoutWrapper(lstm_cell_fw,output_keep_prob=1. - dropout_value)
        
        
        lstm_cell_bw = tf.contrib.rnn.LSTMCell(num_cell,forget_bias=1.0)
        dropout_wrapper_bw = tf.contrib.rnn.DropoutWrapper(lstm_cell_bw,output_keep_prob=1.- dropout_value)
        
        
        output , last_state     = tf.nn.bidirectional_dynamic_rnn(dropout_wrapper_fw,
                                                      dropout_wrapper_bw,
                                                      final_input,
                                                      sequence_length=sequence_length,
                                                      dtype=tf.float32)
        
        logits = tf.concat(output,2)
        #ex . 12 x 10 x 24
        
        
        
        
        #Attention_layer
        
        
        #ex : 12x10x24 . ===> 120 x 24
        input_reshape = tf.reshape(logits,[-1,num_cell*2])
        
        
        #num_cell = 12
        #ex : # 24 x 1
        attention_size = tf.get_variable(name='attention_size',
                                         shape=[2*num_cell,1],
                                         dtype=tf.float32,
                                         initializer=tf.random_uniform_initializer(-0.01,0.01))
        # bias 1
        bias          = tf.get_variable(name='bias',shape=[1],
                                        dtype=tf.float32,
                                        initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
        #projection without activation 
        #ex : 120x24 matmul 24x 1 ==> 120x1
        attention_projection = tf.add(tf.matmul(input_reshape,attention_size),bias)
        
        
        #reshape . 120x1 ==> 12x10x1 (shape of input )
        output_reshape = tf.reshape(attention_projection,[tf.shape(sentence)[0],tf.shape(sentence)[1],-1])
        
        #softmax over logits 12x10x1
        attention_output = tf.nn.softmax(output_reshape,dim=1)
        
        
        #reshape as input 12x10
        attention_visualize = tf.reshape(attention_output,
                                         [tf.shape(sentence)[0],
                                          tf.shape(sentence)[1]],
                                         name='Plot')
        
        
        # 12x10x1 multiply 12x10x24  == > 12x10x24
        attention_projection_output = tf.multiply(attention_output,logits)
        
        #reduce across time 120x10x24 ==> 12x24
        Final_output = tf.reduce_sum(attention_projection_output,1)
        
        
        
        #fully_connected_layer
        
        f_c = tf.get_variable(name='f_c_layer',
                              shape=[2*num_cell,category_size],
                              dtype=tf.float32,
                              initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
        f_bias = tf.get_variable(name='fc_bias',
                                 shape=[category_size],
                                dtype=tf.float32,
                                 initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        output_projection = tf.add(tf.matmul(Final_output,f_c),f_bias)
        
        
        
        probability_dist  = tf.nn.softmax(output_projection,name='pred')
        
        prediction  = tf.argmax(probability_dist,axis=-1)
        
        
        #as_usual_stuff_
        cross_entropy  = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intents,logits=output_projection)
        loss = tf.reduce_mean(cross_entropy)
        accuracy_calculation = tf.reduce_mean(tf.cast(tf.equal(tf.cast(prediction,tf.int32),intents),tf.float32))
        
        
        
        self.output = {  'loss':loss,
                         'accuracy':accuracy_calculation,
                         'prediction':prediction,
                         'probai':probability_dist,
                         'logits':output_projection,
                         'attention_visualize':attention_visualize
            
                       }
        
        
        # this guy is hero of this movie
        self.train = tf.train.AdamOptimizer().minimize(loss)
        

def execute_model(model):
    
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        for i in range(1000):
            outp,train = sess.run([model.output,model.train],feed_dict={model.placeholders['input_d']:np.random.randint(0,3,[12,10]),
                                                model.placeholders['labels']:np.random.randint(0,2,[12,]),
                                                model.placeholders['dropout']:0,
                                                model.placeholders['pos']: np.random.randint(0,3,[12,10]) })
            print(outp['loss'],outp['accuracy'],outp['prediction'].shape,outp['attention_visualize'].shape)
        
        
if __name__ == '__main__':
    
    model = lstm_Attention_with_Pos(130,100,12,0.5,120,22,12)
    execute_model(model)
    
    
    
    
        
        
        
        
        
        
        
        
        
        
        
        
        

2.4834156 0.0 (12,) (12, 10)
2.4823997 0.41666666 (12,) (12, 10)
2.4806073 0.41666666 (12,) (12, 10)
2.4770155 0.5833333 (12,) (12, 10)
2.4765751 0.41666666 (12,) (12, 10)
2.4728703 0.5833333 (12,) (12, 10)
2.4712582 0.5 (12,) (12, 10)
2.4691603 0.5 (12,) (12, 10)
2.4656918 0.5 (12,) (12, 10)
2.4632146 0.5833333 (12,) (12, 10)
2.4597352 0.6666667 (12,) (12, 10)
2.457375 0.41666666 (12,) (12, 10)
2.4543126 0.41666666 (12,) (12, 10)
2.4561393 0.083333336 (12,) (12, 10)
2.4477806 0.33333334 (12,) (12, 10)
2.4430175 0.5833333 (12,) (12, 10)
2.4380262 0.33333334 (12,) (12, 10)
2.4290304 0.75 (12,) (12, 10)
2.426644 0.75 (12,) (12, 10)
2.4208653 0.33333334 (12,) (12, 10)
2.415568 0.5 (12,) (12, 10)
2.4134526 0.5 (12,) (12, 10)
2.4016454 0.41666666 (12,) (12, 10)
2.3904955 0.41666666 (12,) (12, 10)
2.375353 0.16666667 (12,) (12, 10)
2.3651912 0.5 (12,) (12, 10)
2.3558714 0.33333334 (12,) (12, 10)
2.349716 0.41666666 (12,) (12, 10)
2.3360832 0.33333334 (12,) (12, 10)
2.3257353 0.5833333 (12,) 

0.7032633 0.5 (12,) (12, 10)
0.7072544 0.5 (12,) (12, 10)
0.72375154 0.33333334 (12,) (12, 10)
0.70211333 0.5 (12,) (12, 10)
0.6848896 0.75 (12,) (12, 10)
0.70520097 0.5 (12,) (12, 10)
0.7073655 0.5 (12,) (12, 10)
0.73329145 0.33333334 (12,) (12, 10)
0.7176876 0.41666666 (12,) (12, 10)
0.7195719 0.41666666 (12,) (12, 10)
0.69943714 0.5833333 (12,) (12, 10)
0.7091624 0.5 (12,) (12, 10)
0.68501765 0.6666667 (12,) (12, 10)
0.6919754 0.5 (12,) (12, 10)
0.6790693 0.6666667 (12,) (12, 10)
0.7125723 0.5 (12,) (12, 10)
0.7196687 0.41666666 (12,) (12, 10)
0.73398906 0.33333334 (12,) (12, 10)
0.7090152 0.41666666 (12,) (12, 10)
0.7417441 0.16666667 (12,) (12, 10)
0.7128361 0.41666666 (12,) (12, 10)
0.7217758 0.33333334 (12,) (12, 10)
0.70106584 0.6666667 (12,) (12, 10)
0.70462984 0.41666666 (12,) (12, 10)
0.6999838 0.5 (12,) (12, 10)
0.6980384 0.75 (12,) (12, 10)
0.6989024 0.5833333 (12,) (12, 10)
0.7126805 0.25 (12,) (12, 10)
0.6984809 0.6666667 (12,) (12, 10)
0.7117679 0.33333334 (12,) (12, 10

0.70931107 0.25 (12,) (12, 10)
0.6953055 0.33333334 (12,) (12, 10)
0.693984 0.5 (12,) (12, 10)
0.7008009 0.5 (12,) (12, 10)
0.7111876 0.25 (12,) (12, 10)
0.70873183 0.25 (12,) (12, 10)
0.7038638 0.25 (12,) (12, 10)
0.6848586 0.6666667 (12,) (12, 10)
0.6892765 0.5 (12,) (12, 10)
0.7016964 0.5833333 (12,) (12, 10)
0.67972153 0.75 (12,) (12, 10)
0.70801544 0.33333334 (12,) (12, 10)
0.720167 0.16666667 (12,) (12, 10)
0.68634313 0.6666667 (12,) (12, 10)
0.69492036 0.5833333 (12,) (12, 10)
0.6932381 0.5833333 (12,) (12, 10)
0.73411626 0.083333336 (12,) (12, 10)
0.6886837 0.5 (12,) (12, 10)
0.7160581 0.25 (12,) (12, 10)
0.69472855 0.6666667 (12,) (12, 10)
0.70299214 0.41666666 (12,) (12, 10)
0.7011369 0.41666666 (12,) (12, 10)
0.70549434 0.33333334 (12,) (12, 10)
0.6996475 0.41666666 (12,) (12, 10)
0.69767493 0.5833333 (12,) (12, 10)
0.70663196 0.41666666 (12,) (12, 10)
0.6851328 0.6666667 (12,) (12, 10)
0.69623214 0.5 (12,) (12, 10)
0.6795354 0.6666667 (12,) (12, 10)
0.70313066 0.41666666 (1

0.70232123 0.33333334 (12,) (12, 10)
0.68199605 0.75 (12,) (12, 10)
0.7021361 0.41666666 (12,) (12, 10)
0.6992702 0.33333334 (12,) (12, 10)
0.6944639 0.5 (12,) (12, 10)
0.6930222 0.41666666 (12,) (12, 10)
0.6860725 0.75 (12,) (12, 10)
0.6890699 0.6666667 (12,) (12, 10)
0.6818471 0.75 (12,) (12, 10)
0.68425393 0.6666667 (12,) (12, 10)
0.6998784 0.5 (12,) (12, 10)
0.6935417 0.5833333 (12,) (12, 10)
0.68872184 0.41666666 (12,) (12, 10)
0.7058334 0.5833333 (12,) (12, 10)
0.69287986 0.5 (12,) (12, 10)
0.721255 0.33333334 (12,) (12, 10)
0.6688339 0.75 (12,) (12, 10)
0.73316413 0.25 (12,) (12, 10)
0.7214217 0.33333334 (12,) (12, 10)
0.7177817 0.41666666 (12,) (12, 10)
0.70324326 0.5 (12,) (12, 10)
0.6907199 0.5 (12,) (12, 10)
0.6967249 0.41666666 (12,) (12, 10)
0.69632655 0.5 (12,) (12, 10)
0.7158205 0.41666666 (12,) (12, 10)
0.6938529 0.6666667 (12,) (12, 10)
0.6997082 0.5 (12,) (12, 10)
0.7003948 0.41666666 (12,) (12, 10)
0.68182737 0.75 (12,) (12, 10)
0.69861984 0.5 (12,) (12, 10)
0.697691

0.69787294 0.6666667 (12,) (12, 10)
0.6947684 0.5 (12,) (12, 10)
0.70199805 0.25 (12,) (12, 10)
